Step 1: Setup and Data Download


In [2]:
"""
Notebook-cell version: VGG19 transfer learning on Oxford 102 Flowers
- Random split 50/25/25
- Run twice by calling run_experiment(split_seed=1) and run_experiment(split_seed=2)
- Saves curves + checkpoint into out_dir
"""

import json
import math
import os
import random
from dataclasses import dataclass
from typing import Dict, List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset, DataLoader, Dataset, Subset
from torchvision import datasets, models, transforms

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from pathlib import Path
import scipy.io as sio
from torch.utils.data import Dataset
from PIL import Image


In [3]:

class Flowers102Local(Dataset):
    """
    Local Oxford 102 Flowers dataset:
    root/
      jpg/
      imagelabels.mat
      setid.mat
    """
    def __init__(self, root, transform=None):
        self.root = Path(root)
        self.img_dir = self.root / "jpg"
        self.transform = transform

        mat = sio.loadmat(self.root / "imagelabels.mat")
        labels = mat["labels"].squeeze().astype(int)  # 1..102
        self.labels = (labels - 1).tolist()           # 0..101

        self.image_paths = [
            self.img_dir / f"image_{i:05d}.jpg"
            for i in range(1, len(self.labels) + 1)
        ]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        y = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, y


In [5]:
# -----------------------------
# Dataset utilities
# -----------------------------
@dataclass
class SplitIndices:
    train: List[int]
    val: List[int]
    test: List[int]


class TransformOverrideDataset(Dataset):
    def __init__(self, base: Dataset, transform):
        self.base = base
        self.transform = transform

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        x, y = self.base[idx]
        if self.transform is not None:
            x = self.transform(x)
        return x, y



In [6]:
def load_flowers102_all(root: str) -> Dataset:
    return Flowers102Local(root=root, transform=None)


def make_random_split_indices(n: int, split_seed: int) -> SplitIndices:
    g = torch.Generator().manual_seed(split_seed)
    perm = torch.randperm(n, generator=g).tolist()

    n_train = int(0.50 * n)
    n_val = int(0.25 * n)
    n_test = n - n_train - n_val

    train_idx = perm[:n_train]
    val_idx = perm[n_train:n_train + n_val]
    test_idx = perm[n_train + n_val:n_train + n_val + n_test]

    return SplitIndices(train=train_idx, val=val_idx, test=test_idx)


def save_split_indices(path: str, split: SplitIndices) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump({"train": split.train, "val": split.val, "test": split.test}, f)


def load_split_indices(path: str) -> SplitIndices:
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    return SplitIndices(train=obj["train"], val=obj["val"], test=obj["test"])


def build_transforms(img_size: int = 224):
    imagenet_mean = [0.485, 0.456, 0.406]
    imagenet_std = [0.229, 0.224, 0.225]

    train_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.02),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    eval_tf = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
    ])

    return train_tf, eval_tf



In [8]:
def make_loaders(
    root: str,
    batch_size: int,
    num_workers: int,
    split_seed: int,
    split_cache_path: str,
    img_size: int = 224,
) -> Tuple[DataLoader, DataLoader, DataLoader, SplitIndices]:
    base_all = load_flowers102_all(root=root)
    n = len(base_all)

    if os.path.isfile(split_cache_path):
        split = load_split_indices(split_cache_path)
    else:
        split = make_random_split_indices(n=n, split_seed=split_seed)
        save_split_indices(split_cache_path, split)

    train_tf, eval_tf = build_transforms(img_size=img_size)

    train_ds = TransformOverrideDataset(Subset(base_all, split.train), transform=train_tf)
    val_ds   = TransformOverrideDataset(Subset(base_all, split.val), transform=eval_tf)
    test_ds  = TransformOverrideDataset(Subset(base_all, split.test), transform=eval_tf)

    # In Windows notebooks, num_workers>0 can cause issues. Use 0 unless you know it's stable.
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=True)
    val_loader   = DataLoader(val_ds, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)
    test_loader  = DataLoader(test_ds, batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, test_loader, split


In [9]:
# -----------------------------
# Model
# -----------------------------
def build_vgg19_classifier(num_classes: int = 102, freeze_features: bool = True) -> nn.Module:
    model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)

    if freeze_features:
        for p in model.features.parameters():
            p.requires_grad = False

    in_features = model.classifier[-1].in_features
    model.classifier[-1] = nn.Linear(in_features, num_classes)
    return model


In [10]:

# -----------------------------
# Train/Eval loops
# -----------------------------
@torch.no_grad()
def evaluate(model: nn.Module, loader: DataLoader, device: torch.device, criterion: nn.Module) -> Tuple[float, float]:
    model.eval()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        logits = model(x)
        loss = criterion(logits, y)

        total_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    return total_loss / max(1, total), correct / max(1, total)



In [11]:
def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device,
    criterion: nn.Module,
    optimizer: optim.Optimizer,
) -> Tuple[float, float]:
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    return total_loss / max(1, total), correct / max(1, total)



In [12]:
def save_curves(out_dir: str, history: Dict[str, List[float]]) -> None:
    os.makedirs(out_dir, exist_ok=True)

    plt.figure()
    plt.plot(history["train_acc"], label="train")
    plt.plot(history["val_acc"], label="val")
    plt.plot(history["test_acc"], label="test")
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "accuracy_vgg19.png"), dpi=160)
    plt.close()

    plt.figure()
    plt.plot(history["train_loss"], label="train")
    plt.plot(history["val_loss"], label="val")
    plt.plot(history["test_loss"], label="test")
    plt.xlabel("epoch")
    plt.ylabel("cross_entropy")
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, "loss_vgg19.png"), dpi=160)
    plt.close()

    with open(os.path.join(out_dir, "history_vgg19.json"), "w", encoding="utf-8") as f:
        json.dump(history, f, indent=2)


In [14]:

def save_checkpoint(path: str, model: nn.Module, optimizer: optim.Optimizer, epoch: int, best_val_acc: float) -> None:
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(
        {"epoch": epoch, "model_state": model.state_dict(),
         "optimizer_state": optimizer.state_dict(), "best_val_acc": best_val_acc},
        path,
    )


@torch.no_grad()
def predict_proba(model: nn.Module, images: torch.Tensor, device: torch.device) -> torch.Tensor:
    model.eval()
    images = images.to(device)
    logits = model(images)
    return torch.softmax(logits, dim=1).cpu()



In [15]:

# -----------------------------
#  "main"
# -----------------------------
def run_experiment(
    data_root: str = "./data",
    out_dir: str = "./results/vgg19_seed1",
    epochs: int = 35,
    batch_size: int = 32,
    lr: float = 1e-4,
    weight_decay: float = 0.0,
    num_workers: int = 0,         # NOTE: 0 is safest in Windows notebooks
    img_size: int = 224,
    split_seed: int = 1,
    freeze_features: bool = True,
    early_stop_patience: int = 7,
    device: str = "cuda",
):
    if device == "cuda" and not torch.cuda.is_available():
        print("CUDA requested but not available. Falling back to CPU.")
        device = "cpu"
    device_t = torch.device(device)

    os.makedirs(out_dir, exist_ok=True)
    split_cache_path = os.path.join(out_dir, f"split_indices_seed_{split_seed}.json")

    set_seed(split_seed)

    train_loader, val_loader, test_loader, _split = make_loaders(
        root=data_root,
        batch_size=batch_size,
        num_workers=num_workers,
        split_seed=split_seed,
        split_cache_path=split_cache_path,
        img_size=img_size,
    )

    model = build_vgg19_classifier(num_classes=102, freeze_features=freeze_features).to(device_t)
    trainable_params = [p for p in model.parameters() if p.requires_grad]

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(trainable_params, lr=lr, weight_decay=weight_decay)

    history = {k: [] for k in ["train_loss","val_loss","test_loss","train_acc","val_acc","test_acc"]}

    best_val_acc = -1.0
    best_val_loss = math.inf
    epochs_no_improve = 0
    ckpt_path = os.path.join(out_dir, "best_vgg19.pt")

    for epoch in range(1, epochs + 1):
        tr_loss, tr_acc = train_one_epoch(model, train_loader, device_t, criterion, optimizer)
        va_loss, va_acc = evaluate(model, val_loader, device_t, criterion)
        te_loss, te_acc = evaluate(model, test_loader, device_t, criterion)

        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)
        history["test_loss"].append(te_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(va_acc)
        history["test_acc"].append(te_acc)

        print(
            f"Epoch {epoch:03d}/{epochs} | "
            f"train loss {tr_loss:.4f} acc {tr_acc:.4f} | "
            f"val loss {va_loss:.4f} acc {va_acc:.4f} | "
            f"test loss {te_loss:.4f} acc {te_acc:.4f}"
        )

        if va_acc > best_val_acc:
            best_val_acc = va_acc
            save_checkpoint(ckpt_path, model, optimizer, epoch, best_val_acc)

        if va_loss < best_val_loss - 1e-6:
            best_val_loss = va_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= early_stop_patience:
                print(f"Early stopping triggered (patience={early_stop_patience}).")
                break

    save_curves(out_dir, history)

    best_epoch = int(np.argmax(history["val_acc"])) + 1
    print("\nSummary")
    print(f"- Split seed: {split_seed}")
    print(f"- Best val acc: {max(history['val_acc']):.4f} (epoch {best_epoch})")
    print(f"- Last test acc: {history['test_acc'][-1]:.4f}")
    print(f"- Saved: {os.path.join(out_dir, 'accuracy_vgg19.png')}")
    print(f"- Saved: {os.path.join(out_dir, 'loss_vgg19.png')}")
    print(f"- Checkpoint: {ckpt_path}")
    print(f"- Split indices: {split_cache_path}")

    return history, ckpt_path


In [ ]:
import os
import random
import numpy as np
import torch

def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    # Determinism (can reduce speed; )
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [19]:
h1, ckpt1 = run_experiment(
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=1,
    out_dir="./results/vgg19_seed1"
)


Epoch 001/35 | train loss 3.1448 acc 0.2865 | val loss 1.5717 acc 0.6116 | test loss 1.6156 acc 0.6099
Epoch 002/35 | train loss 1.3604 acc 0.6412 | val loss 0.9499 acc 0.7538 | test loss 0.9626 acc 0.7480
Epoch 003/35 | train loss 0.8579 acc 0.7587 | val loss 0.7738 acc 0.7831 | test loss 0.7844 acc 0.7871
Epoch 004/35 | train loss 0.6324 acc 0.8190 | val loss 0.6457 acc 0.8217 | test loss 0.6614 acc 0.8193
Epoch 005/35 | train loss 0.4731 acc 0.8625 | val loss 0.6027 acc 0.8334 | test loss 0.6209 acc 0.8330
Epoch 006/35 | train loss 0.3598 acc 0.8898 | val loss 0.5875 acc 0.8315 | test loss 0.6244 acc 0.8384
Epoch 007/35 | train loss 0.3036 acc 0.9108 | val loss 0.5879 acc 0.8344 | test loss 0.6061 acc 0.8364
Epoch 008/35 | train loss 0.2374 acc 0.9287 | val loss 0.5396 acc 0.8520 | test loss 0.5580 acc 0.8540
Epoch 009/35 | train loss 0.2295 acc 0.9304 | val loss 0.5612 acc 0.8447 | test loss 0.5477 acc 0.8579
Epoch 010/35 | train loss 0.1890 acc 0.9436 | val loss 0.5072 acc 0.8500 

In [20]:
h2, ckpt2 = run_experiment(
    data_root=r"C:\Users\hp\Downloads\102flowers",
    split_seed=2,
    out_dir="./results/vgg19_seed2"
)

Epoch 001/35 | train loss 3.1209 acc 0.2985 | val loss 1.5443 acc 0.6082 | test loss 1.4719 acc 0.6323
Epoch 002/35 | train loss 1.3546 acc 0.6290 | val loss 0.9866 acc 0.7479 | test loss 0.9165 acc 0.7559
Epoch 003/35 | train loss 0.8602 acc 0.7653 | val loss 0.8094 acc 0.7816 | test loss 0.7834 acc 0.7856
Epoch 004/35 | train loss 0.6295 acc 0.8227 | val loss 0.7129 acc 0.8046 | test loss 0.6548 acc 0.8174
Epoch 005/35 | train loss 0.4379 acc 0.8708 | val loss 0.6384 acc 0.8241 | test loss 0.5684 acc 0.8403
Epoch 006/35 | train loss 0.3623 acc 0.8896 | val loss 0.6342 acc 0.8295 | test loss 0.5392 acc 0.8550
Epoch 007/35 | train loss 0.3023 acc 0.9135 | val loss 0.6117 acc 0.8349 | test loss 0.5510 acc 0.8452
Epoch 008/35 | train loss 0.2088 acc 0.9387 | val loss 0.5598 acc 0.8549 | test loss 0.4834 acc 0.8662
Epoch 009/35 | train loss 0.2258 acc 0.9255 | val loss 0.6057 acc 0.8398 | test loss 0.5124 acc 0.8540
Epoch 010/35 | train loss 0.1719 acc 0.9489 | val loss 0.5469 acc 0.8534 